In [1]:
# These two lines are necessary only if GemPy is not installed
import sys, os
sys.path.append("../..")
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN, device=cuda"

# Importing GemPy
import gempy as gp

# Embedding matplotlib figures in the notebooks
%matplotlib inline

# Importing auxiliary libraries
import numpy as np
import matplotlib.pyplot as plt
# sys.path = list(np.insert(sys.path, 0, "../../../pyvista"))
import pyvista as pv

Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
b'/tmp/try_flags_vchrih3v.c:4:10: fatal error: cudnn.h: No such file or directory\n #include <cudnn.h>\n          ^~~~~~~~~\ncompilation terminated.\n'
Mapped name None to device cuda: GeForce RTX 2070 (0000:01:00.0)


In [2]:
geo_model = gp.load_model(
    'Tutorial_ch1-9b_Fault_relations', 
    path= '../data/gempy_models', 
    recompile=False
)

gp.set_interpolation_data(
    geo_model, 
    output='geology', 
    compile_theano=True, 
    theano_optimizer='fast_run', # fast_compile, fast_run
    dtype="float32",  # float64 for model stability
    verbose=[]
)

gp.compute_model(geo_model, compute_mesh=True)

Active grids: ['regular']
Compiling theano function...
Level of Optimization:  fast_run
Device:  cuda
Precision:  float32
Number of faults:  2
Compilation Done!


/home/alex/anaconda3/envs/topology/lib/python3.7/site-packages/theano/tensor/slinalg.py:255: LinAlgWarning: Ill-conditioned matrix (rcond=2.46753e-12): result may not be accurate.
  rval = scipy.linalg.solve(A, b)



Lithology ids 
  [7. 7. 7. ... 3. 3. 3.] 

In [3]:
from gempy.plot import vista
from importlib import reload
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [4]:
geo_model.surfaces.df

,surface,series,order_surfaces,isBasement,color,vertices,edges,id
7,fault2,fault_series2,1,False,#015482,"[[354.9399948120117, 0.0, -980.0], [344.933433...","[[2, 1, 0], [3, 1, 2], [0, 5, 4], [0, 4, 2], [...",1
5,fault1,fault_series1,1,False,#9f0052,"[[14.601644277572632, 20.0, -60.0], [20.0, 0.0...","[[2, 1, 0], [0, 1, 3], [2, 5, 4], [2, 0, 5], [...",2
6,rock4,series1,1,False,#ffbe00,"[[0.0, 0.0, -560.0016403198242], [0.0, 20.0, -...","[[2, 1, 0], [3, 1, 2], [3, 4, 1], [5, 4, 3], [...",3
0,rock3,series1,2,False,#728f02,"[[0.0, 0.0, -610.0020217895508], [0.0, 20.0, -...","[[2, 1, 0], [3, 1, 2], [3, 4, 1], [5, 4, 3], [...",4
3,rock2,series1,3,False,#443988,"[[0.0, 0.0, -660.0025939941406], [0.0, 20.0, -...","[[2, 1, 0], [3, 1, 2], [3, 4, 1], [5, 4, 3], [...",5
1,rock1,series1,4,False,#ff3f20,"[[0.0, 0.0, -710.0033187866211], [0.0, 20.0, -...","[[2, 1, 0], [3, 1, 2], [3, 4, 1], [5, 4, 3], [...",6
4,basement,basement_series,1,True,#325916,NaN,NaN,7


In [73]:
p = pv.Plotter(notebook=False)


def callback(*args):
    for arg in args:
        print(arg)
    
p.add_mesh(
    pv.PolyData(
        np.random.randn(300).reshape(-1, 3)
    )
)

# p.add_line_widget(
#     callback,
#     use_vertices=True,
#     center=100
    
# )
p.add_plane_widget(
    callback, 
    factor=0.1,

)

p.show()

(1.0, 0.0, 0.0)
(-0.09553716081475572, -0.02832828197865478, -0.6839474577709088)
(1.0, 0.0, 0.0)
(0.10712827514246809, -0.02832828197865478, -0.6839474577709088)


[(0.7678066735310811, 1.5096288742184405, 0.10078508353611916),
 (-0.09553716081475572, -0.02832828197865478, -0.6839474577709088),
 (-0.19124649850221231, -0.35880582128410876, 0.9136099602255218)]

In [17]:
geo_model.modify_orientations?

In [24]:
reload(vista)
gpv = vista.Vista(geo_model, plotter_type="basic", **dict(notebook=False))

gpv.set_bounds()
# gpv.plot_surface_points("rock4")
# gpv.plot_surface_points_all()
# gpv.plot_surface_points_interactive("rock1", radius=20)
# gpv.plot_surface_points_interactive_all(radius=15)
# gpv.plot_orientations("rock4")

# gpv.plot_surface("rock4")
# gpv.plot_surfaces_all()

# gpv.plot_orientations_all()

# gpv.plot_orientations_interactive("rock3")
gpv.plot_surfaces_all()
gpv.plot_orientations_interactive_all()
gpv._live_updating = True



# gpv.plot_structured_grid("lith")
# gpv.plot_structured_grqid("scalar")



gpv.p.show()

/home/alex/anaconda3/envs/topology/lib/python3.7/site-packages/theano/tensor/slinalg.py:255: LinAlgWarning: Ill-conditioned matrix (rcond=2.46753e-12): result may not be accurate.
  rval = scipy.linalg.solve(A, b)


[(3575.6570606696723, 1517.6421097363664, -91.12414129787138),
 (490.0, 490.0, -510.0),
 (-0.0778754001258435, -0.1668091246320309, 0.982909018167366)]

In [17]:
pv.Sphere?

In [43]:
hasattr(actor, "points")

True

In [14]:
actor = pv.PolyData(np.ones((3, 3)))

In [18]:
actors = [pv.PolyData(np.ones((3, 3))).points.tostring() for i in range(5)]

In [20]:
actor.points.tostring() in actors

True

In [21]:
actor.points.tostring()

b'\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?'

In [122]:
actor = pv.PolyData(np.ones((3, 3)))
actors = [actor]
actors_hash = [hash(actor)]


In [123]:
actor in actors

True

In [124]:
pv.PolyData(np.ones((3, 3))) in actors

False

In [125]:
hash(pv.PolyData(np.ones((3, 3)))) in actors_hash

False

In [126]:
hash(actor) in actors_hash

True

dict

In [ ]:
import pyvista as pv
from nptyping import Array


class PyvistaPlotter:
    def __init__(self, geo_model, plotter_kwargs={}):
        self.geo_model = geo_model
        self.p = pv.Plotter(**plotter_kwargs)
        self.colors = {
            fmt:c for fmt, c in zip(geo_model.surfaces.df.surface.values, 
                                    geo_model.surfaces.df.color.values)
        }
        
        self._surface_points = False
        self._orientations = False
        self._surfaces = False
        
        self.formations = geo_model.surfaces.df.surface[:-1].values
    
    def add_surface_points(self, fmt:str, mesh_kwargs={}):
        i = self.geo_model.surface_points.df.groupby("surface").groups[fmt]
        self.p.add_mesh(
            pv.PolyData(
                self.geo_model.surface_points.df.loc[i][["X", "Y", "Z"]].values
            ),
            color=self.colors[fmt],
            **mesh_kwargs
        )
        
    def add_surface_points_widget(self, fmt:str):
        i = self.geo_model.surface_points.df.groupby("surface").groups[fmt]
        
        for _, pt in self.geo_model.surface_points.df.loc[i].iterrows():
            self.p.add_sphere_widget(
                callback,
                center=pt[["X", "Y", "Z"]]
            )
    
    def add_surface_orientations(self, fmt:str, mesh_kwargs={}):
        i = self.geo_model.orientations.df.groupby("surface").groups[fmt]
        pts = self.geo_model.orientations.df.loc[i][["X", "Y", "Z"]].values
        nrms = self.geo_model.orientations.df.loc[i][["G_x", "G_y", "G_z"]].values
        for pt, nrm in zip(pts, nrms):
            self.p.add_mesh(
                pv.Line(pointa=pt, pointb=pt+200*nrm),
                color=self.colors[fmt],
                **mesh_kwargs
            )
            
    def add_surface(self, fmt:str, mesh_kwargs={}):
        i = np.where(self.geo_model.surfaces.df.surface == fmt)[0][0] 
        ver = self.geo_model.solutions.vertices[i]
        
        sim = self._simplices_to_pv_tri_simplices(
            self.geo_model.solutions.edges[i]
        )
        
        self.p.add_mesh(
            pv.PolyData(ver, sim),
            color=self.colors[fmt],
            **mesh_kwargs
        )
            
    def add_surfaces(self, mesh_kwargs={}):
        if self._surfaces:
            return
        vertices = self.geo_model.solutions.vertices
        simplices = self.geo_model.solutions.edges
        for fmt in geo_model.surfaces.df.surface[:-1].values:
            self.add_surface(fmt, mesh_kwargs=mesh_kwargs)
        self._surfaces = True
        
    def add_surfaces_points(self, mesh_kwargs={}):
        if self._surface_points:
            return
        for fmt in self.formations:
            self.add_surface_points(fmt, mesh_kwargs=mesh_kwargs)
        self._surface_points = True
            
    def add_surfaces_orientations(self, mesh_kwargs={}):
        if self._orientations:
            return
        for fmt in self.formations:
            self.add_surface_orientations(fmt, mesh_kwargs=mesh_kwargs)
        self._orientations = True    
        
    def _simplices_to_pv_tri_simplices(self, sim:Array[int, ..., 3]) -> Array[int, ..., 4]:
        """Convert triangle simplices (n, 3) to pyvista-compatible
        simplices (n, 4)."""
        n_edges = np.ones(sim.shape[0]) * 3
        return np.append(n_edges[:, None], sim, axis=1)


#### Point picking

In [24]:
hash(pv.PolyData(np.ones((3,3)))) == hash(pv.PolyData(np.ones((3,3))))

True

In [ ]:
def callback(polydata, pid):
    print(pid)

p.enable_point_picking(callback, use_mesh=True) 

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


C:\Users\r02as18\miniconda3\envs\topology\lib\site-packages\theano\gpuarray\dnn.py:184: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to a version >= v5 and <= v7.
  warnings.warn("Your cuDNN version is more recent than "


Active grids: ['regular']
Compiling theano function...
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float64
Number of faults:  2
Compilation Done!


In [7]:
# gp.plot.plot_section(geo_model, show_data=True)


Lithology ids 
  [7. 7. 7. ... 3. 3. 3.] 

In [3]:
import pyvista as pv
from PyQt5 import Qt
import sys
import numpy as np
from nptyping import Array


class MainWindow(Qt.QMainWindow):
    def __init__(self, geo_model, parent=None, show=True):
        Qt.QMainWindow.__init__(self, parent)
        self.geo_model = geo_model
        
        self.colors = self.geo_model.surfaces.df.color.values
        
        # create the frame
        self.frame = Qt.QFrame()
        vlayout = Qt.QVBoxLayout()

        # add the pyvista interactor object
        self.vtk_widget = pv.QtInteractor(self.frame)
        vlayout.addWidget(self.vtk_widget)

        self.frame.setLayout(vlayout)
        self.setCentralWidget(self.frame)

        # menu
        mainMenu = self.menuBar()
        fileMenu = mainMenu.addMenu('File')
        exitButton = Qt.QAction('Exit', self)
        exitButton.setShortcut('Ctrl+Q')
        exitButton.triggered.connect(self.close)
        fileMenu.addAction(exitButton)

        # allow adding a sphere
        meshMenu = mainMenu.addMenu('Plot')
        self.add_surfacepoints_action = Qt.QAction('Add Surface Points', self)
        self.add_surfacepoints_action.triggered.connect(self.add_surface_points)
        meshMenu.addAction(self.add_surfacepoints_action)
        
        self.add_surfaces_action = Qt.QAction('Add Surfaces', self)
        self.add_surfaces_action.triggered.connect(self.add_surfaces)
        meshMenu.addAction(self.add_surfaces_action)
        
        self.vtk_widget.enable_point_picking() 

        if show:
            self.show()
        
    def add_surface_points(self):
        points = pv.PolyData(
            self.geo_model.surface_points.df[["X", "Y", "Z"]].values
        )
        self.vtk_widget.add_mesh(points)
        
    def add_surfaces(self):
        vertices = self.geo_model.solutions.vertices
        simplices = self.geo_model.solutions.edges
        
        for i, (ver, sim) in enumerate(zip(self.geo_model.solutions.vertices, 
                            self.geo_model.solutions.edges)):
            sim = self._simplices_to_pv_tri_simplices(sim)
            
            self.vtk_widget.add_mesh(pv.PolyData(ver, sim), color=self.colors[i])
            
    def _simplices_to_pv_tri_simplices(self, sim:Array[int, ..., 3]) -> Array[int, ..., 4]:
        """Convert triangle simplices (n, 3) to pyvista-compatible
        simplices (n, 4)."""
        n_edges = np.ones(sim.shape[0]) * 3
        return np.append(n_edges[:, None], sim, axis=1)


In [6]:
from PyQt5 import QtCore
from PyQt5.QtWidgets import QWidget

In [ ]:
%gui qt

app = QtCore.QCoreApplication.instance()
if app is None:
    app = Qt.QApplication(sys.argv)
    
w = QWidget()
w.setWindowTitle('Simple')
w.show()
window = MainWindow(geo_model)
# sys.exit(app.exec_())
app.exec_()

In [3]:
from gempy.plot import vista
from importlib import reload
reload(vista)

<module 'gempy.plot.vista' from '../..\\gempy\\plot\\vista.py'>

In [4]:
import pyvista as pv

In [1]:
from create_geomodel_gullfaks import create_geomodel

ModuleNotFoundError: No module named 'create_geomodel_gullfaks'

In [5]:
gv = vista.Vista(geo_model, notebook=False, plotter_type="background")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [40]:
gv.plot_surfaces()

,val
7,PolyData (0x23e0d48e468)\n N Cells:\t7154\n ...
5,PolyData (0x23e0d48e6a8)\n N Cells:\t12152\n ...
6,PolyData (0x23e0d48eb28)\n N Cells:\t5824\n ...
0,PolyData (0x23e0d48ea08)\n N Cells:\t5684\n ...
3,PolyData (0x23e0d3b7a68)\n N Cells:\t5770\n ...
1,PolyData (0x23e0d3b7a08)\n N Cells:\t5684\n ...


In [115]:
class Vista2:
    def __init__(self, geo_model, **kwargs):
        self.geo_model = geo_model
        self.p = pv.BackgroundPlotter(**kwargs)
        self.entities = []
        
    def plot_surfaces(self):
        for idx, val in self.geo_model.surfaces.df.dropna().iterrows():
            if idx in self.entities:
                continue
                
            surf = pv.PolyData(
                val.vertices, 
                np.insert(val['edges'], 0, 3, axis=1).ravel()
            )
            self.p.add_mesh(surf, color=val['color'])
            self.entities.append(idx)
            

gv2 = Vista2(geo_model)

In [116]:
gv2.plot_surfaces()